In [1]:
import pandas as pd
import numpy as np
#data_path = "July2021_for_test.xlsx"
data_path = "20_80_plain.xlsx"
df = pd.read_excel(data_path, usecols="A:Q", engine="openpyxl")
#print(df["Плавок(окр)"].sum())
df

,ID Плавки,imax,ПОРТ,Марка,АПК,АЦВ,У-9,Номер толщины(H_i)
0,0,20,Калининград,S355JR,1,0,0,1
1,1,20,Калининград,S355JR,1,0,0,1
2,2,20,Калининград,S355JR,1,0,0,1
3,3,20,Калининград,S355JR,1,0,0,1
4,4,20,Калининград,S355JR,1,0,0,1
...,...,...,...,...,...,...,...,...
75,75,20,Туапсе,1026,1,0,0,1
76,76,20,Туапсе,1026,1,0,0,1
77,77,20,Туапсе,1026,1,0,0,1
78,78,20,Туапсе,1026,1,0,0,1


In [2]:
# Array for thicknesses by number of heat
w_numbers = df["Номер толщины(H_i)"].to_numpy()
# Array for imax
j_imax = df["imax"].to_numpy() 

In [3]:
from amplify import (
    BinaryPoly,
    BinaryQuadraticModel,
    sum_poly,pair_sum,
    gen_symbols,
    Solver,
    decode_solution,
)
from amplify import Solver
from amplify.client import FixstarsClient
#from amplify.client.ocean import DWaveSamplerClient
from amplify.constraint import less_equal, equal_to, penalty,one_hot

def flatten(t):
    return [item for sublist in t for item in sublist]

client = FixstarsClient()
client.token = "LvfkonaT7sLymsCqHyyQUp7JxHlo02CF"
client.parameters.timeout = 60000  # Timeout is 60 seconds

solver = Solver(client)

In [4]:
#n = 80 #Number of timeslots for 5 days
n = 20
#m = 795 #Number of heats
m = 80
w = 6 #Number of thicknesses
w_max = [3,4,0,1,1,1] #Max thicknesses per UNRS

x = gen_symbols(BinaryPoly,m,n) #Variable x - heat for timeslot

In [5]:
# Condition for min(n, imax[j])
for j in range (m):
    if (df.loc[j, "imax"]<n):
        for i in range (df.loc[j, "imax"], n):
            x[j][i]=0

In [6]:
# В одном слоте 4 плавки по кол-ву УНРС
# Либо для 5.1. Constraints on columns
col_constraints = [
    equal_to(sum_poly([x[j][i] for j in range(m)]), 4) for i in range(n)
]

In [7]:
H1_constr = sum(col_constraints)

In [8]:
# 5.2 Deadline
S = [int(df.loc[j,"imax"]<=n) for j in range(m)] #плавка должна быть разлита обязательно - взять из df

In [9]:
H2_0 = sum_poly(m, lambda j: S[j]*(1 - sum_poly(n, lambda i:(x[j][i])))**2)

In [10]:
# Либо для 5.2 H2_1 Constraints on rows
H2_1_constr = 0
row_constraints = 0
for j in range(m):
    if (1-S[j]):
        row_constraints += less_equal(sum_poly([x[j][i] for i in range(n)]), 1)

#H2_1_constr = row_constraints
H2_1_constr = sum([less_equal(x[0][0],1)]) if not row_constraints else sum(row_constraints)

In [11]:
# 5.3 Ограничение на количество плавок одной толщины в каждом ts
w_sum = [[0 for x in range(w)] for y in range(n)] #array for sums of each thickness for each ts
#constr_w = 0
# Number of heats for each thickness for each ts
for i in range(n):
    for w_ind in range(w):
        for j in range(m):
            w_sum[i][w_ind]+= (int(w_numbers[j]==w_ind))*x[j][i]
        #constr_w += less_equal(w_sum[i][w_ind], w_max[w_ind], label=label)       

In [12]:
#Create array x_w_full for slack variables
x_w_full = []
x_w_start = n*m
#for i in range(n):
for w_ind in range(w):
    w_max_ind = w_max[w_ind]+1
    x_w_full.append(gen_symbols(BinaryPoly, x_w_start, (w_max_ind,n)))
    x_w_start += w_max_ind*n       

In [13]:
# <============5.3 
H3 = 0
x_w_start = n*m
for i in range (n):
    for w_ind in range(w):
        H3_i_w = 0
        H3_i_w += w_sum[i][w_ind]
        for ww in range (1, w_max[w_ind]+1):
            x_w_start += ww
            H3_i_w +=ww*x_w_full[w_ind][ww][i]
        x_w_start += w_max[w_ind]*n    
        H3_i_w = (H3_i_w - w_max[w_ind])**2 
        H3+=H3_i_w   

In [ ]:
# H4 APK(L) ACV(V)
"""
K = 1 #Coeff
L = []
V = [] # TBD
H4 = 0
for j in range (m):
    L.append(df.loc[j,"АПК"])   
for i in range(n):
    H4 += less_equal(sum_poly(m, lambda j:(L[j]*x[j][i])), 2)
H4 = K*sum(H4)  
"""

In [ ]:
# 5.5 Логистические расходы
"""
F = 1 #Coeff
H5 = F*(sum_poly(m, lambda j: (j_imax[j] - sum_poly(n, lambda i:(i*x[j][i])))**2))
"""

In [14]:
# Array for offset of slack variables (distance - for count numbers of heats with different thicknesses) 
D_w = [0 for x in range(w)]
for ww in range(w):
    D_w[ww] = [[abs(y-x) for x in range(w_max[ww]+1)] for y in range(w_max[ww]+1)]

In [15]:
# 5.5 Перенастройки по толщине
# Для каждой толщины считаем, сколько плавок данной толщины делалось в соседних ts. Минимизируем дельту
# We can use array w_sum = [[0 for x in range(w)] for y in range(n)]

H_5_i_0 = 0
for w_ind in range(w):
    for i in range(n):
        #H_5_i +=(1-sum_poly(w_max[w_ind]+1,lambda ww: (x_w_full[w_ind][ww][i])))**2 
        H_5_i_0 +=one_hot(sum_poly(w_max[w_ind]+1,lambda ww: (x_w_full[w_ind][ww][i])))
    
H_5_i_1 = 0    
for i in range(n-1):
    for w_ind in range(w):
        for e_ind in range(w_max[w_ind]+1):
            for f_ind in range(w_max[w_ind]+1):
                H_5_i_1+= 0.5*D_w[w_ind][e_ind][f_ind]*x_w_full[w_ind][e_ind][i]*x_w_full[w_ind][f_ind][i+1]       

In [ ]:
#H = H5 + H2_0 + H1_constr + H2_1_constr + H3 + H4 + H6

In [16]:
# Сoefficients for model  
A=9
B=9
E=9
K=9
R=9
Q1=9
Q2=1

model = Q2*H_5_i_1 + E*H3 + B*H2_0 + A*H1_constr + R*H2_1_constr + Q1*H_5_i_0
#model = H2_0 + H3 + H1_constr + H2_1_constr

In [17]:
model.num_logical_vars

1920

In [18]:
count = 0
for count in range (1):
    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values

    x_values = decode_solution(x, values)

    print(energy)
    
    x_values_array = np.asarray(x_values, dtype = np.int32)
    np.savetxt("slabs-"+str(pd.Timestamp.now())+".csv", x_values_array, fmt='%i', delimiter=",")
    
    count+=1

0.0


In [ ]:
for i in range (n):
    print("Timeslot ", i)
    for j in range(m):
        if (x_values[j][i]):
            print("Heat ", j)
            print ("Thickness ", w_numbers[j])
            print (x_values[j][i])

In [ ]:
client_result = solver.client_result
client_result.annealing_time_ms

In [ ]:
client_result.timing.cpu_time

In [ ]:
def get_assignments(x_ij, rng=n):
    casts_assign = [[] for y in range(rng)]
    cast = 0
    for c in x_ij:
        if 1 in c:
            casts_assign[list(c).index(1)].append(cast)
        cast += 1
    return casts_assign

def get_thickness_by_cast_num(c):
    return df["Номер толщины(H_i)"][c]

#xij = np.loadtxt(open('/home/eugene/__Work/Slabs/Code/slabs-2021-11-09 00 01 03.398912.csv', "r"), delimiter=",")  # or just take the decoded solution 2D-array

casts_assign = get_assignments(x_values)
print(casts_assign)

thickness_assign = [list(map(get_thickness_by_cast_num, slot)) for slot in casts_assign]
print(thickness_assign)

In [ ]:
def check_thickness_assign(d):
    if d[0] > 3:
        print('Thickness 0 casts number violation')
    if d[1] > 4:
        print('Thickness 1 casts number violation')
    if d[2] > 0:
        print('Thickness 2 casts number violation')
    if d[3] > 1:
        print('Thickness 3 casts number violation')
    if d[4] > 1:
        print('Thickness 4 casts number violation')
    if d[5] > 1:
        print('Thickness 5 casts number violation')
    else:
        print("Ok")

slot_num = 0
for slot in thickness_assign:
    d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    for thickness in slot:
        d[thickness] += 1
    print(str(d) + 'in slot ' + str(slot_num))
    check_thickness_assign(d)

    slot_num += 1

In [ ]:
# Calculation for UNRS changeovers
slot_num = 0
d_array = []
d_array_diff = []
for slot in thickness_assign:
    d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    for thickness in slot:
        d[thickness] += 1
    d_array.append(d)
    slot_num += 1
    
count = 0   # number of changeovers 
for i in range(n-1):
    slot_a = d_array[i]
    slot_b = d_array[i+1]
    diff = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
    if(slot_a != slot_b):
        print("Change, slots:", i,"and ", i+1)
        for thickness in slot_b:
            if(slot_b[thickness] - slot_a[thickness]!=0):
                diff[thickness] = slot_b[thickness] - slot_a[thickness]
                count +=0.5 
        print(diff)    
print(count) 

In [ ]:
""""
#Debug
x_w_full_flat = [item for sublist in x_w_full for item in sublist]
x_w_full_flat_ = [item for sublist in x_w_full_flat for item in sublist]

#Extract from values
q_val = [0 for y in range(len(x_w_full_flat_))]
ii = 0
for q_var in x_w_full_flat_:
    ind = int(str(q_var)[3:])
    q_val[ii] = values[ind]
    ii+=1

#Create empty array x_w_full_copy for slack variables
x_w_full_copy = []
x_w_start = n*m
w_max_ind = 0
for w_ind in range(w):
    w_max_ind = w_max[w_ind]+1
    x_w_full_copy.append([[0 for v in range (n)] for w in range (w_max_ind)])
    x_w_start += w_max_ind*n

#Fill slack var values
ii = 0
for index, item in enumerate(x_w_full_copy):
    for index,item0 in enumerate(item):
        for index,item1 in enumerate(item0):
            if (q_val[ii]):
                #print(q_val[ii])
                item0[index] = q_val[ii]
            ii+=1

In [ ]:
"""
H_5_i_0 = 0
for w_ind in range(w):
    for i in range(n):
        H_5_i_w = 0
        #H_5_i_w +=(1-sum_poly(w_max[w_ind]+1,lambda ww: (x_w_full[w_ind][ww][i])))**2
        for ww in range (w_max[w_ind]+1):
            H_5_i_w +=x_w_full_copy[w_ind][ww][i]
        print(H_5_i_w)    
    H_5_i_0 += H_5_i_w
    
H_5_i_1 = 0    
for i in range(n-1):
    for w_ind in range(w):
        H_5_i_1_w = 0 #for each w
        for e_ind in range(w_max[w_ind]+1):
            for f_ind in range(w_max[w_ind]+1):
                H_5_i_1_w+= 0.5*D_w[w_ind][e_ind][f_ind]*x_w_full_copy[w_ind][e_ind][i]*x_w_full_copy[w_ind][f_ind][i+1]
        H_5_i_1 += H_5_i_1_w

In [ ]:
#H_5_i_0

In [ ]:
#H_5_i_1